In [72]:
from Pipeline import Helper, Purchase_Probabilities
import pandas as pd

In [73]:
DATA_PATH   =   r'C:/Users/Benedikt/Documents/GitHub/mlim-g2/data/'

EXPORT_PATH =   r'C:/Users/Benedikt/Documents/GitHub/mlim-g2/data/export'

files = {
    'data': [
        DATA_PATH + 'baskets.parquet',
        #DATA_PATH + 'coupon_index.parquet', 
        DATA_PATH + 'coupons.parquet', 
        DATA_PATH + 'merged.parquet',
        DATA_PATH + 'clean.parquet', 
        #DATA_PATH + 'prepare.parquet'
    ]}

In [74]:
df = Purchase_Probabilities()

NameError: name 'basket' is not defined

In [ ]:
class Elasticities(Helper):
        
        """    
        class inheritance level: (inheritance levels: 0 -> 1 -> 2 -> 3)
            - 1
        goal:
            - calculate week specific price elasticities
        functionality:
            - load and merge last week of training window from baskets & coupons df
            - calculate product specific price elasticities from normal price & 30% discounts
        """
    
    
    def __init__(self):
        """
        attributes: 
            - get_elasticities
            - (helper-attributes: data, mappings)

        public methods:
            - get_elasticities: pull and merge dataframes, return price elasticities

            - (helper: load, dump, reduce_data_size, get_merged_clean, get_mappings)
        """
        super().__init__()
        self.data['baskets'] = basket
        self.data['coupons'] = coupons


    def get_elasticities():
         """       
            use:
                - calculate weekly price elasticities for whole dataset

            requirements: baskets.parquet, coupons.parquet stored on disc

            input:
                - baskets: pd.DataFrame
                - coupons: pd.DataFrame

            return: pd.DataFrame
                - elasticities df (week x product)
            """

        start = time.time()
        elasticities = []
        total_basket_count = 100000

        for i in basket['week'].unique():

            basket_week = basket[basket['week'] == i].copy()
            coupons_week = coupons[coupons['week'] == i].copy()
            basket_week = basket_week.merge(coupons_week, how = "outer")
            basket_week['discount'] = basket_week['discount'].fillna(0)
            basket_week['price'] = basket_week['price'].fillna(0)
            elast = []

            for i in range(250):
                reg_price = basket_week[basket_week['product'] == i]
                reg_price_buy = len(reg_price[reg_price['discount'] == 0])
                all_discounts_offers = len(reg_price[reg_price['discount'] > 0])
                reg_price_offer = total_basket_count - all_discounts_offers
                reg_price_buy_rate = reg_price_buy / reg_price_offer
                discount_30 = reg_price[reg_price['discount'] == 30]
                discount_30_offer = len(discount_30)
                discount_30_buy = (discount_30['price'] != 0).sum()
                discount_buy_rate = discount_30_buy / discount_30_offer
                elast.append((discount_buy_rate - reg_price_buy_rate) / (0.3 * reg_price_buy_rate))

            elasticities.append(elast)

        end = time.time()
        print(end - start)
        return pd.DataFrame(elasticities).T

In [20]:
class Redemptions(Helper):
    
    """    
    class inheritance level: (inheritance levels: 0 -> 1 -> 2 -> 3)
        - 1
    goal:
        - calculate costumer and product specific coupon redemption rates
    functionality:
        - take merged dataframe
        - calculate product and shopper_product coupon redemption rates
        - calculate products that have only been bought because of a discount    
    """
    
    def __init__(self):
        """
        attributes: 
            - (helper-attributes: data, mappings)

        public methods:
            - get_reduced_coupons: prepare coupons df
            - product_redemption: calculate product_redemption_likelihood 
            - costumer_redemption: calculate shopper_product_redemption_likelihood
            - only_discount_buy: calculate costumer specific products that were only bought because of a discount

            - (helper: load, dump, reduce_data_size, get_merged_clean, get_mappings)
        """    
        super().__init__()
        self.data['clean'] = output
        self.data['reduced_coupons'] = None
        
        
    def get_reduced_coupons(self, df='clean'):
        """       
        use:
            - prepare coupons df for use in class
            
        requirements:
            - 'clean' df stored in class

        input:
            - 'clean': pd.DataFrame
                - merged and cleaned dataframe

        return: 
            - pd.DataFrame
                - coupon data extracted
        """
        coupons = output[output['discount'] > 0].copy()
        coupons.loc[(coupons['price'].notnull()), 'redeemed'] = 1
        coupons['redeemed'] = coupons['redeemed'].fillna(0)
        
        return self.data['reduced_coupons']
        
        
    def product_redemption(self, df='clean'):
        """       
        use:
            - calculate product specific redemption rate from prepared dataframe
            
        requirements:
            - 'clean' df stored in class

        input:
            - 'clean': pd.DataFrame
                - merged and cleaned dataframe

        return: pd.GroupBy
        """
        redemption_rate = self.data['reduced_coupons'].groupby(['product'])['redeemed'].mean()#
        
        return redemption_rate

    def costumer_redemption(self, df='clean'):
        """       
        use:
            - calculate costumer specific redemption rate from prepared dataframe
            
        requirements:
            - 'clean' df stored in class

        input:
            - 'clean': pd.DataFrame
                - merged and cleaned dataframe

        return: pd.GroupBy
        """
        costumer_redemption_rate = self.data['reduced_coupons'].groupby(['shopper', 'product'])['redeemed'].mean()

        return costumer_redemption_rate 
    
    def only_discount_buy(self, df='clean'):
        """       
        use:
            - calculate costumer specific products that were only bought because of a discount from prepared dataframe
            
        requirements:
            - 'clean' df stored in class

        input:
            - 'clean': pd.DataFrame
                - merged and cleaned dataframe

        return: pd.GroupBy
        """
        buy_all = output.groupby(['shopper', 'product']).size()
        discount = self.data['reduced_coupons'].groupby(['shopper', 'product']).size()
        discount_buy = discount / buy_all
        discount_buy = discount_buy.fillna(0)
        discount_buy = discount_buy.rename('discount_buy')

        return discount_buy